# Get the Exact Answer
Start off by computing the exact Hessian to use a reference point. 
First relax the structure then compute the Hessians using [ase's Vibrations module](https://databases.fysik.dtu.dk/ase/ase/vibrations/modes.html), which will compute them numerically using central derivatives

In [ ]:
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import VibrationsData, Vibrations
from ase.calculators.mopac import MOPAC
from ase.calculators.psi4 import Psi4
from ase.optimize import BFGS
from ase import Atoms, units
from ase.io import write, read
from jitterbug.utils import make_calculator
from contextlib import redirect_stderr
from time import perf_counter
from platform import node
from pathlib import Path
from os import devnull
import numpy as np
import shutil
import json
import os

Configuration

In [ ]:
molecule_name = 'caffeine'
relax_method = 'pm7/None'  # Method used to relax geometry 
hess_method = None  # Method used to perform Hessian computation, None to use same
basis = None  # Set to None for MOPAC methods
threads = min(os.cpu_count(), 12)
delta = 0.01

Derived

In [ ]:
relax_method, relax_basis = relax_method.split("/")
if hess_method is None:
    hess_method, hess_basis = relax_method, relax_basis
else:
    hess_method, hess_basis = hess_method.split("/")

In [ ]:
run_name = f'{molecule_name}_{hess_method}_{hess_basis}_at_{relax_method}_{relax_basis}'
run_name_with_delta = f'{run_name}_d={delta:.3g}'
out_dir = Path('data') / 'exact'
if (out_dir / f'{run_name_with_delta}-times.json').exists():
    raise ValueError('Already done!')
print(f'Run name: {run_name_with_delta}')

## Load in Target Molecule
We have it in a JSON file from PubChem

In [ ]:
def load_molecule(name: str) -> Atoms:
    """Load a molecule from a PubChem JSON file
    
    Args:
        name: Name of the molecule
    Returns:
        ASE Atoms object
    """
    
    # Get the compound data
    with open(f'data/structures/{name}.json') as fp:
        data = json.load(fp)
    data = data['PC_Compounds'][0]
        
    # Extract data from the JSON
    atomic_numbers = data['atoms']['element']
    positions = np.zeros((len(atomic_numbers), 3))
    conf_data = data['coords'][0]['conformers'][0]
    for i, c in enumerate('xyz'):
        if c in conf_data:
            positions[:, i] = conf_data[c]
        
    # Build the object    
    return Atoms(numbers=atomic_numbers, positions=positions)

In [ ]:
atoms = load_molecule(molecule_name)

## Perform the Geometry Optimization
Build the ASE calculator then run QuasiNewton to a high tolerance

In [ ]:
calc = make_calculator(relax_method, relax_basis, num_threads=threads)

Either relax or load the existing molecule

In [ ]:
geom_path = out_dir / f'{molecule_name}_{relax_method}_{relax_basis}.xyz'
print(f'Geometry path: {geom_path}')

In [ ]:
%%time
if geom_path.exists():
    atoms = read(geom_path)
    atoms.calc = calc
else:
    atoms.calc = calc
    dyn = BFGS(atoms)
    with redirect_stderr(devnull):
        dyn.run(fmax=0.01)

Save the output file

In [ ]:
out_dir.mkdir(exist_ok=True)

In [ ]:
write(geom_path, atoms)

## Compute the Hessian using ASE
ASE has a built-in method which uses finite displacements

Make the calculator for the hessian

In [ ]:
calc = make_calculator(hess_method, hess_basis, num_threads=threads)
atoms.calc = calc

Perform the computation

In [ ]:
if Path('vib').is_dir():
    shutil.rmtree('vib')

In [ ]:
%%time
finite_diff_time = perf_counter()
vib = Vibrations(atoms, delta=delta)
vib.run()
finite_diff_time = perf_counter() - finite_diff_time

Save the vibration data

In [ ]:
vib_data = vib.get_vibrations()
with (out_dir / f'{run_name_with_delta}-ase.json').open('w') as fp:
    vib_data.write(fp)

Print the ZPE for reference

In [ ]:
vib_data.get_zero_point_energy()

## Repeat with Psi4's analytic derivatives
See if we get the same answer faster

In [ ]:
psi4_path = out_dir / f'{run_name}-psi4.json'
if isinstance(calc, Psi4) and "cc" not in hess_method and not psi4_path.exists():
    # Compute
    analytic_time = perf_counter()
    calc.set_psi4(atoms)
    hess = calc.psi4.hessian(f'{hess_method}/{hess_basis}')
    analytic_time = perf_counter() - analytic_time

    # Convert to ASE format
    analytic_hess = hess.to_array() * units.Hartree / units.Bohr / units.Bohr
    vib_data = VibrationsData.from_2d(atoms, analytic_hess)
    with psi4_path.open('w') as fp:
        vib_data.write(fp)
else:
    analytic_time = None

Save the runtimes

In [ ]:
with (out_dir / f'{run_name_with_delta}-times.json').open('w') as fp:
    json.dump({
        'hostname': node(),
        'finite-diff': finite_diff_time,
        'analytic': analytic_time,
    }, fp)